In [2]:
import pandas as pd

import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName1').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)


In [148]:
# Functionality for computing features
from pyspark.ml import feature
# Functionality for regression
from pyspark.ml import regression
# Funcionality for classification
from pyspark.ml import classification
# Object for creating sequences of transformations
from pyspark.ml import Pipeline,evaluation
from pyspark.ml.classification import LogisticRegression

In [3]:
from pyspark.sql import functions as fn

In [4]:
tv=pd.read_csv('C:/Users/mihir/Desktop/sem2/IST718/dataset/TV_Commercials_Cleaned.csv')

In [5]:
tv.shape

(129685, 205)

In [6]:
tv_narm=tv.loc[:,((tv.isnull().sum()/tv.shape[0])*100)<25]

In [7]:
tv_narm.shape

(129685, 100)

In [8]:
tv_narm.columns

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25',
       '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37',
       '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49',
       '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61',
       '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73',
       '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '92',
       '102', '103', '104', '107', '117', '118', '119', '491', '572', '573',
       '623', '816', '4124', '4125', 'Label'],
      dtype='object')

In [29]:
from missingpy import KNNImputer
imputer = KNNImputer()
tv_imputed = imputer.fit(tv_narm)

C:\Users\mihir\Anaconda3\lib\site-packages\missingpy\knnimpute.py:224: UserWarning: There are rows with more than 50.0% missing values. These rows are not included as donor neighbors.
  .format(self.row_max_missing * 100))


In [31]:
tv_imputed.transform(tv_narm)

C:\Users\mihir\Anaconda3\lib\site-packages\missingpy\knnimpute.py:282: UserWarning: There are rows with more than 50.0% missing values. The missing features in these rows are imputed with column means.
  .format(self.row_max_missing * 100))


MemoryError: 

In [9]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import Imputer
import numpy as np
import pandas as pd

class MiceImputer:
    
    model_dict_ = {}
    
    def __init__(self, seed_nulls=False, seed_strategy='mean'):
        self.seed_nulls = seed_nulls
        self.seed_strategy = seed_strategy
        
    
    def transform(self, X):
        col_order = X.columns
        new_X = []
        mutate_cols = list(self.model_dict_.keys())
        
        for i in mutate_cols:
            y = X[i]
            x_null = X[y.isnull()]
            y_null = y[y.isnull()].reset_index()['index']
            y_notnull = y[y.notnull()]
            
            model = self.model_dict_.get(i)
            
            if self.seed_nulls:
                x_null = model[1].transform(x_null)
            else:
                null_check = x_null.isnull().any()
                x_null = x_null[null_check.index[~null_check]]
            
            pred = pd.concat([pd.Series(model[0].predict(x_null))\
                              .to_frame()\
                              .set_index(y_null),y_notnull], axis=0)\
                              .rename(columns={0: i})
            
            new_X.append(pred)

        new_X.append(X[X.columns.difference(mutate_cols)])

        final = pd.concat(new_X, axis=1)[col_order]

        return final
        
        
    def fit(self, X):      
        x = X.fillna(value=np.nan)

        null_check = x.isnull().any()
        null_data = x[null_check.index[null_check]]
        
        for i in null_data:
            y = null_data[i]
            y_notnull = y[y.notnull()]

            model_list = []
            if self.seed_nulls:
                imp = Imputer(strategy=self.seed_strategy)
                model_list.append(imp.fit(x))
                non_null_data = pd.DataFrame(imp.fit_transform(x))
                
            else:
                non_null_data = x[null_check.index[~null_check]]
                
            
            x_notnull = non_null_data[y.notnull()]
            
            if y.nunique() > 2:
                model = LinearRegression()
                model.fit(x_notnull, y_notnull)
                model_list.insert(0, model)
                self.model_dict_.update({i: model_list})
            else:
                model = LogisticRegression()
                model.fit(x_notnull, y_notnull)
                model_list.insert(0, model)
                self.model_dict_.update({i: model_list})

        return self
        

    def fit_transform(self, X):
        return self.fit(X).transform(X)

In [10]:
miceimpute=MiceImputer()

In [86]:
imputed_tv=miceimpute.fit_transform(tv_narm)

In [87]:
sum(imputed_tv.isnull().sum()!=0)


0

In [88]:
import pickle

In [89]:
objdict=open('C:/Users/mihir/Desktop/sem2/IST718/tv_pickle1.p','rb')


In [90]:
namedict=pickle.load(objdict)

In [92]:
namedict

{1: 'Shot Length',
 2: 'Motion Distribution Mean',
 3: 'Motion Distribution Variance',
 4: 'Frame Difference Distribution Mean',
 5: 'Frame Difference Distribution Variance',
 6: 'Short time energy Mean',
 7: 'Short time energy Variance',
 8: 'ZCR Mean',
 9: 'ZCR Variance',
 10: 'SpectralCentroid Mean',
 11: 'SpectralCentroid Variance',
 12: 'Spectral Roll off Mean',
 13: 'Spectral Roll off Variance',
 14: 'Spectral Flux Mean',
 15: 'Spectral Flux Variance',
 16: 'Fundamental Frequecy Mean',
 17: 'Fundamental Frequecy Variance',
 4124: 'Edge Change Ratio Mean',
 4125: 'Edge Change Ratio Variance',
 18: 'Motion Distribution bin_18',
 19: 'Motion Distribution bin_19',
 20: 'Motion Distribution bin_20',
 21: 'Motion Distribution bin_21',
 22: 'Motion Distribution bin_22',
 23: 'Motion Distribution bin_23',
 24: 'Motion Distribution bin_24',
 25: 'Motion Distribution bin_25',
 26: 'Motion Distribution bin_26',
 27: 'Motion Distribution bin_27',
 28: 'Motion Distribution bin_28',
 29: 'Moti

In [93]:
cols=imputed_tv.columns
cols=[int(i) for i in cols[:-1]]
colnames=[namedict[i] for i in cols]
colnames.append('Label')

In [94]:
colnames

['Shot Length',
 'Motion Distribution Mean',
 'Motion Distribution Variance',
 'Frame Difference Distribution Mean',
 'Frame Difference Distribution Variance',
 'Short time energy Mean',
 'Short time energy Variance',
 'ZCR Mean',
 'ZCR Variance',
 'SpectralCentroid Mean',
 'SpectralCentroid Variance',
 'Spectral Roll off Mean',
 'Spectral Roll off Variance',
 'Spectral Flux Mean',
 'Spectral Flux Variance',
 'Fundamental Frequecy Mean',
 'Fundamental Frequecy Variance',
 'Motion Distribution bin_18',
 'Motion Distribution bin_19',
 'Motion Distribution bin_20',
 'Motion Distribution bin_21',
 'Motion Distribution bin_22',
 'Motion Distribution bin_23',
 'Motion Distribution bin_24',
 'Motion Distribution bin_25',
 'Motion Distribution bin_26',
 'Motion Distribution bin_27',
 'Motion Distribution bin_28',
 'Motion Distribution bin_29',
 'Motion Distribution bin_30',
 'Motion Distribution bin_31',
 'Motion Distribution bin_32',
 'Motion Distribution bin_33',
 'Motion Distribution bin_34

In [95]:
imputed_tv.columns=colnames

In [96]:
imputed_tv.head()

,Shot Length,Motion Distribution Mean,Motion Distribution Variance,Frame Difference Distribution Mean,Frame Difference Distribution Variance,Short time energy Mean,Short time energy Variance,ZCR Mean,ZCR Variance,SpectralCentroid Mean,...,Text Area Distribution bin_118,Text Area Distribution bin_119,Bag Audio Words bin_491,Bag Audio Words bin_572,Bag Audio Words bin_573,Bag Audio Words bin_623,Bag Audio Words bin_816,Edge Change Ratio Mean,Edge Change Ratio Variance,Label
0,123,1.316440,1.516003,5.605905,5.346760,0.013233,0.010729,0.091743,0.050768,3808.067871,...,0.251688,0.176144,0.069915,0.141949,0.103814,0.050847,0.036017,0.422334,0.663918,1
1,124,0.966079,0.546420,4.046537,3.190973,0.008338,0.011490,0.075504,0.065841,3466.266113,...,0.206926,0.051774,0.075630,0.012605,0.037815,0.086134,0.023109,0.332664,0.766184,1
2,109,2.035407,0.571643,9.551406,5.803685,0.015189,0.014294,0.094209,0.044991,3798.196533,...,0.185767,0.038852,0.069712,0.038462,0.088942,0.139423,0.052885,0.346674,0.225022,1
3,86,3.206008,0.786326,10.092709,2.693058,0.013962,0.011039,0.092042,0.043756,3761.712402,...,0.165600,0.003983,0.135802,0.012346,0.086420,0.212963,0.077160,0.993323,0.840083,1
4,76,3.135861,0.896346,10.348035,2.651010,0.020914,0.012061,0.108018,0.052617,3784.488037,...,0.159495,0.000066,0.021127,0.007042,0.154930,0.151408,0.017606,0.341520,0.710470,1


In [72]:
cor = imputed_tv.corr()

In [105]:
####COPIED#####
# Create correlation matrix
corr_matrix = imputed_tv.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
imputed_tv_reduced=imputed_tv.drop(imputed_tv[to_drop], axis=1)

In [106]:
to_drop

['Spectral Roll off Mean',
 'Spectral Roll off Variance',
 'Motion Distribution bin_28',
 'Motion Distribution bin_30',
 'Motion Distribution bin_34',
 'Motion Distribution bin_37',
 'Motion Distribution bin_40',
 'Motion Distribution bin_47',
 'Motion Distribution bin_48',
 'Motion Distribution bin_49',
 'Motion Distribution bin_50',
 'Motion Distribution bin_51',
 'Motion Distribution bin_52',
 'Motion Distribution bin_57',
 'Frame Difference bin_64',
 'Frame Difference bin_65',
 'Frame Difference bin_66',
 'Frame Difference bin_67',
 'Frame Difference bin_68',
 'Frame Difference bin_69',
 'Frame Difference bin_72',
 'Frame Difference bin_73',
 'Frame Difference bin_74',
 'Frame Difference bin_75',
 'Frame Difference bin_77',
 'Frame Difference bin_78']

In [107]:
imputed_tv_reduced.shape

(129685, 74)

In [129]:
imputed_tv_reduced['Edge Change Ratio Mean'].head()

0    0.422334
1    0.332664
2    0.346674
3    0.993323
4    0.341520
Name: Edge Change Ratio Mean, dtype: float64

In [111]:
tv_spark_df = sqlContext.createDataFrame(imputed_tv_reduced)

In [114]:
tv_spark_df.columns

['Shot Length',
 'Motion Distribution Mean',
 'Motion Distribution Variance',
 'Frame Difference Distribution Mean',
 'Frame Difference Distribution Variance',
 'Short time energy Mean',
 'Short time energy Variance',
 'ZCR Mean',
 'ZCR Variance',
 'SpectralCentroid Mean',
 'SpectralCentroid Variance',
 'Spectral Flux Mean',
 'Spectral Flux Variance',
 'Fundamental Frequecy Mean',
 'Fundamental Frequecy Variance',
 'Motion Distribution bin_18',
 'Motion Distribution bin_19',
 'Motion Distribution bin_20',
 'Motion Distribution bin_21',
 'Motion Distribution bin_22',
 'Motion Distribution bin_23',
 'Motion Distribution bin_24',
 'Motion Distribution bin_25',
 'Motion Distribution bin_26',
 'Motion Distribution bin_27',
 'Motion Distribution bin_29',
 'Motion Distribution bin_31',
 'Motion Distribution bin_32',
 'Motion Distribution bin_33',
 'Motion Distribution bin_35',
 'Motion Distribution bin_36',
 'Motion Distribution bin_38',
 'Motion Distribution bin_39',
 'Motion Distribution bi

In [115]:
tv_spark_df.schema

StructType(List(StructField(Shot Length,LongType,true),StructField(Motion Distribution Mean,DoubleType,true),StructField(Motion Distribution Variance,DoubleType,true),StructField(Frame Difference Distribution Mean,DoubleType,true),StructField(Frame Difference Distribution Variance,DoubleType,true),StructField(Short time energy Mean,DoubleType,true),StructField(Short time energy Variance,DoubleType,true),StructField(ZCR Mean,DoubleType,true),StructField(ZCR Variance,DoubleType,true),StructField(SpectralCentroid Mean,DoubleType,true),StructField(SpectralCentroid Variance,DoubleType,true),StructField(Spectral Flux Mean,DoubleType,true),StructField(Spectral Flux Variance,DoubleType,true),StructField(Fundamental Frequecy Mean,DoubleType,true),StructField(Fundamental Frequecy Variance,DoubleType,true),StructField(Motion Distribution bin_18,DoubleType,true),StructField(Motion Distribution bin_19,DoubleType,true),StructField(Motion Distribution bin_20,DoubleType,true),StructField(Motion Distri

In [127]:
tv_spark_df.groupBy('Label').agg(fn.count('Label')).show()

+-----+------------+
|Label|count(Label)|
+-----+------------+
|  0.0|       47454|
|  1.0|       82231|
+-----+------------+



In [126]:
tv_spark_df=tv_spark_df.withColumn('Label',fn.when(fn.col('Label')==-1,0.).otherwise(1))

In [133]:
feat_list=tv_spark_df.columns[:-1]


In [134]:
###Cross validation
training_df, validation_df, testing_df = tv_spark_df.randomSplit([0.6, 0.3, 0.1], seed=0)

In [141]:
#feature assembler and scaling  pipeline 
pipe1tv=Pipeline(stages=[feature.VectorAssembler(inputCols=feat_list,outputCol='features'),feature.StandardScaler(inputCol='features',outputCol='scaled',withMean=True)])

In [185]:
#Logistic Regression Pipeline with
lr = LogisticRegression(regParam=0.1, elasticNetParam=0.4,featuresCol='scaled',labelCol='Label')
lr_pipe=Pipeline(stages=[pipe1tv,lr]).fit(training_df)
lr_est=Pipeline(stages=[pipe1tv,lr])

In [147]:
valid_df=lr_pipe.transform(validation_df)

In [151]:
#evaluation
evaluator = evaluation.BinaryClassificationEvaluator(labelCol='Label')

In [154]:
AUC=evaluator.evaluate(valid_df)
AUC

0.8708349937490328

In [164]:
non_zerobool=[i!=0 for i in lr_pipe.stages[-1].coefficients.toArray().tolist()]

In [165]:
from itertools import compress
list(compress(feat_list,non_zerobool))

['Shot Length',
 'Short time energy Mean',
 'Short time energy Variance',
 'SpectralCentroid Variance',
 'Spectral Flux Variance',
 'Motion Distribution bin_58',
 'Frame Difference bin_59',
 'Text Area Distribution bin_104',
 'Text Area Distribution bin_119',
 'Bag Audio Words bin_572',
 'Bag Audio Words bin_573',
 'Bag Audio Words bin_816']

Features:['Shot Length',
 'Short time energy Mean',
 'Short time energy Variance',
 'SpectralCentroid Variance',
 'Spectral Flux Variance',
 'Motion Distribution bin_58',
 'Frame Difference bin_59',
 'Text Area Distribution bin_104',
 'Text Area Distribution bin_119',
 'Bag Audio Words bin_572',
 'Bag Audio Words bin_573',
 'Bag Audio Words bin_816']

In [168]:
#Tune grid 
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [193]:
import numpy as np
paramlist=np.arange(0.1,0.9, 0.1).tolist()

In [194]:
grid = ParamGridBuilder().\
    addGrid(lr_pipe.stages[-1].regParam, paramlist).\
    addGrid(lr_pipe.stages[-1].elasticNetParam, paramlist).\
    build()

In [195]:
all_models = []
for j in range(len(grid)):
    print("Fitting model {}".format(j+1))
    model = lr_est.fit(training_df, grid[j])
    all_models.append(model)

Fitting model 1
Fitting model 2
Fitting model 3
Fitting model 4
Fitting model 5
Fitting model 6
Fitting model 7
Fitting model 8
Fitting model 9
Fitting model 10
Fitting model 11
Fitting model 12
Fitting model 13
Fitting model 14
Fitting model 15
Fitting model 16
Fitting model 17
Fitting model 18
Fitting model 19
Fitting model 20
Fitting model 21
Fitting model 22
Fitting model 23
Fitting model 24
Fitting model 25
Fitting model 26
Fitting model 27
Fitting model 28
Fitting model 29
Fitting model 30
Fitting model 31
Fitting model 32
Fitting model 33
Fitting model 34
Fitting model 35
Fitting model 36
Fitting model 37
Fitting model 38
Fitting model 39
Fitting model 40
Fitting model 41
Fitting model 42
Fitting model 43
Fitting model 44
Fitting model 45
Fitting model 46
Fitting model 47
Fitting model 48
Fitting model 49
Fitting model 50
Fitting model 51
Fitting model 52
Fitting model 53
Fitting model 54
Fitting model 55
Fitting model 56
Fitting model 57
Fitting model 58
Fitting model 59
Fittin

In [201]:
AUC_scores=[evaluator.evaluate(m.transform(validation_df)) for m in all_models]

In [202]:
best_model_idx = np.argmax(AUC_scores)

In [203]:
grid[best_model_idx]

{Param(parent='LogisticRegression_d65adf14706f', name='regParam', doc='regularization parameter (>= 0)'): 0.1,
 Param(parent='LogisticRegression_d65adf14706f', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty'): 0.1}

In [204]:
best_model=all_models[best_model_idx]

In [205]:
AUC_scores[best_model_idx]

0.8856261875259545

In [206]:
#testing AUC
evaluator.evaluate(best_model.transform(testing_df))

0.8815894828798078

In [207]:
non_zerobool=[i!=0 for i in best_model.stages[-1].coefficients.toArray().tolist()]
list(compress(feat_list,non_zerobool))

['Shot Length',
 'Motion Distribution Variance',
 'Frame Difference Distribution Mean',
 'Short time energy Mean',
 'Short time energy Variance',
 'SpectralCentroid Variance',
 'Spectral Flux Variance',
 'Fundamental Frequecy Mean',
 'Fundamental Frequecy Variance',
 'Motion Distribution bin_19',
 'Motion Distribution bin_27',
 'Motion Distribution bin_29',
 'Motion Distribution bin_35',
 'Motion Distribution bin_58',
 'Frame Difference bin_59',
 'Frame Difference bin_62',
 'Frame Difference bin_63',
 'Frame Difference bin_70',
 'Frame Difference bin_76',
 'Frame Difference bin_79',
 'Text Area Distribution bin_92',
 'Text Area Distribution bin_103',
 'Text Area Distribution bin_104',
 'Text Area Distribution bin_119',
 'Bag Audio Words bin_491',
 'Bag Audio Words bin_572',
 'Bag Audio Words bin_573',
 'Bag Audio Words bin_623',
 'Bag Audio Words bin_816']

Features:

['Shot Length',
 'Motion Distribution Variance',
 'Frame Difference Distribution Mean',
 'Short time energy Mean',
 'Short time energy Variance',
 'SpectralCentroid Variance',
 'Spectral Flux Variance',
 'Fundamental Frequecy Mean',
 'Fundamental Frequecy Variance',
 'Motion Distribution bin_19',
 'Motion Distribution bin_27',
 'Motion Distribution bin_29',
 'Motion Distribution bin_35',
 'Motion Distribution bin_58',
 'Frame Difference bin_59',
 'Frame Difference bin_62',
 'Frame Difference bin_63',
 'Frame Difference bin_70',
 'Frame Difference bin_76',
 'Frame Difference bin_79',
 'Text Area Distribution bin_92',
 'Text Area Distribution bin_103',
 'Text Area Distribution bin_104',
 'Text Area Distribution bin_119',
 'Bag Audio Words bin_491',
 'Bag Audio Words bin_572',
 'Bag Audio Words bin_573',
 'Bag Audio Words bin_623',
 'Bag Audio Words bin_816']
 
 
 params:
 
 {Param(parent='LogisticRegression_d65adf14706f', name='regParam', doc='regularization parameter (>= 0)'): 0.1,
 Param(parent='LogisticRegression_d65adf14706f', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty'): 0.1}

In [208]:
len(list(compress(feat_list,non_zerobool)))

29

In [227]:
object_dictionary={'pandas_dataframe_full':imputed_tv,'pandas_dataframe_corrred':imputed_tv_reduced}

In [228]:
import pickle
pickle.dump(object_dictionary,open('pd_dataframes.p','wb'))

In [224]:
best_model.save('bestmodel')

In [226]:
lr_pipe.save('lrmodel')